## Preprocessing

### Imports libraries

In [2]:
import os
import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")

from tqdm import tqdm
from datetime import datetime
import shutil
import dicom2nifti

import brain_mets_classification.config as config
import brain_mets_classification.custom_funcs as funcs

### Copies files and creates new folders with the desired MRI sequences already converted renamed and converted into nifti files

Sequences that will be used for this project:
- axiale T2
- axiale Flair
- axiale T1 vor + nach KM
- sagittale T1 MPRage nach KM
- evtl. DWI (Diffsionswichtung) + ADC

In [4]:
# path to the sample n=30 directory
sampleDirectoryPath = f"{config.path}/RgbBrainMetsSampleN30"

# Creates a new folder within in directory for all the files of the desired MRI sequences
now = datetime.now()
timeFormatted = now.strftime("%Y-%m-%d %H:%M:%S")
pathToImportantSequences = f"{config.path}/Rgb_Brain_Mets_Imp_Sequences_{timeFormatted}"
os.mkdir(pathToImportantSequences)

# list all the names of the folders within the directory
patientFolders = os.listdir(sampleDirectoryPath)

 # removes the .DS_Store file if it can be found within the directory
dsStore = ".DS_Store"
if dsStore in patientFolders:
    patientFolders.remove(dsStore)

patientsIDs = []
#print(patientFolders)


for folder in tqdm(patientFolders): # loops through the all folders for each patient

    mriSequences = []

    patientID = folder.split(" - ")[1]

    # adds the patientID to the list of patientIDs if is hasn't been added before
    if patientID not in patientsIDs:
        patientsIDs.append(patientID)
    
    folderPath = os.path.join(sampleDirectoryPath, folder) #path to the content of each patient's folder
    os.chdir(folderPath)
    mriImagesFolders = os.listdir(os.getcwd())

    for count in range(0, len(mriImagesFolders)):
        mriImagesFolder = mriImagesFolders[count]
        if mriImagesFolder != config.dsStore:
            os.chdir(mriImagesFolder)
        else:
            continue   
        

        mriSequences = os.listdir(os.getcwd())
        # print(mriSequences)

        for sequence in mriSequences:

            #print(sequence)

            sequenceLC = sequence.lower()
            if any(whiteListSeq in sequenceLC for whiteListSeq in config.seq_whitelist) and not any(blackListSeq in sequenceLC for blackListSeq in config.seq_blacklist):
                # sequence fits both the whiteList as well as the blackList

                # create folder for patient
                funcs.createFolderForPatient(path=pathToImportantSequences, patientID=patientID)
                
                pathToSequence = os.path.join(folderPath, mriImagesFolder, sequence)

                #T1/T1CE
                if any(sequence in sequenceLC for sequence in config.T1list) and not any(subSequence in sequenceLC for subSequence in config.sub):
                    
                    # check if T1 is contrast enhanced (CE)
                    if any(ceSequence in sequenceLC for ceSequence in config.KMlist):
                        dicom2nifti.convert_directory(pathToSequence, f"{pathToImportantSequences}/{patientID}")

                        unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToImportantSequences}/{patientID}")
                        os.rename(unRenamedFile, f"{pathToImportantSequences}/{patientID}/{patientID}_T1CE.nii.gz")

                        # niftiFile = os.listdir(f"{pathToImportantSequences}/{patientID}")[0]
                        # os.rename(f"{pathToImportantSequences}/{patientID}/{niftiFile}", f"{pathToImportantSequences}/{patientID}/{patientID}_T1CE.nii.gz")
                        
                        # convert 2 nift, call the file [PATIENTID]_T1CE and move to Patient Folder

                    else:
                        dicom2nifti.convert_directory(pathToSequence, f"{pathToImportantSequences}/{patientID}")
                        unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToImportantSequences}/{patientID}")
                        os.rename(unRenamedFile, f"{pathToImportantSequences}/{patientID}/{patientID}_T1.nii.gz")
                    # convert 2 nift and call the file [PATIENTID]_T1

                #T2
                if any(T2Sequence in sequenceLC for T2Sequence in config.T2list) and not any(T2SternSequence in sequenceLC for T2SternSequence in config.T2STERNlist) and not any(FlairSequence in sequenceLC for FlairSequence in config.FLAIRlist):
                    dicom2nifti.convert_directory(pathToSequence, f"{pathToImportantSequences}/{patientID}")
                    unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToImportantSequences}/{patientID}")
                    os.rename(unRenamedFile, f"{pathToImportantSequences}/{patientID}/{patientID}_T2.nii.gz")

                #FLAIR
                if any(FlairSequence in sequenceLC for FlairSequence in config.FLAIRlist):
                    dicom2nifti.convert_directory(pathToSequence, f"{pathToImportantSequences}/{patientID}")
                    unRenamedFile = funcs.getUnrenamedFile(path=f"{pathToImportantSequences}/{patientID}")
                    os.rename(unRenamedFile, f"{pathToImportantSequences}/{patientID}/{patientID}_FLAIR.nii.gz")



                ## To-do:
                # find the sequences that we actually want
                # dicom2nifti.convert_directory(path_to_dicom_series, path_to_save_nifti_file)





                
                # print(pathToSequence)
                # shutil.copytree(pathToSequence, f"{pathToImportantSequences}/{patientID}", dirs_exist_ok=True)

                ## To-do:
                # copy files into new folder

        # move back out
        os.chdir(folderPath)

print("done")

  0%|          | 0/30 [00:00<?, ?it/s]Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series
Removing duplicate slice from series


done
